# Ingestar datos Instituto Nacional de Estadística

Este ejemplo muestra cómo podemos ingestar datos de la Encuesta de Población Activa y Encuesta Contínua de Hogares

### 1. Encuesta de Población Activa

Este conjunto de datos ha sido descargado de la web del [INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&menu=resultados&secc=1254736030639&idp=1254735976595#!tabs-1254736030639)
en nuestro bucket de S3:

    s3://mitramiss/EPA

It contains data in JSON format about United States legislators and the seats they have held
in the the House of Representatives and the Senate.

### 2. Encuesta Contínua de Hogares

Este conjunto de datos ha sido descargado de la web del [INE](https://www.ine.es/dyngs/INEbase/en/operacion.htm?c=Estadistica_C&cid=1254736176952&menu=resultados&secc=1254736195203&idp=1254735572981#!tabs-1254736195203)
en nuestro bucket de S3:

    s3://mitramiss/ECHHogares
    s3://mitramiss/ECHPersonas

Empezaremos importando las librerías de AWS GLUE.


In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1595925451260_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3. Comprobación de los esquemas que el crawler ha identificado

A continuación visualizaremos el esquema de la tabla `echhogares` mediante las siguientes líneas:

In [3]:
echhogares_dynamicframe = glueContext.create_dynamic_frame.from_catalog(
       database = "epa",
       table_name = "echhogares")
echhogares_dynamicframe.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- id_viv: long
|-- periodo: long
|-- tamano: choice
|    |-- long
|    |-- string
|-- idq_pv: long
|-- ca: long
|-- faccal: double
|-- regvi: long
|-- cocina: long
|-- aseos: long
|-- comedores: long
|-- dormitorios: long
|-- trasteros: long
|-- otrashab: long
|-- habvi: long
|-- metrosvi: long
|-- tipoviv: long
|-- anedi: long
|-- anoanedi: choice
|    |-- long
|    |-- string
|-- fedi: long
|-- tamtoho: long
|-- densidadvi: double
|-- tipoho: choice
|    |-- long
|    |-- string
|-- nacho: long
|-- nucleofam: choice
|    |-- long
|    |-- string
|-- nhijopar: string
|-- nhijo: string
|-- nhijomenor: string

In [5]:
echhogares_res = echhogares_dynamicframe.resolveChoice(specs = [('tamano','cast:long'),('anoanedi','cast:long'),
                                                             ('tipoho','cast:long'),('nucleofam','cast:long')])
echhogares_res.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- id_viv: long
|-- periodo: long
|-- tamano: long
|-- idq_pv: long
|-- ca: long
|-- faccal: double
|-- regvi: long
|-- cocina: long
|-- aseos: long
|-- comedores: long
|-- dormitorios: long
|-- trasteros: long
|-- otrashab: long
|-- habvi: long
|-- metrosvi: long
|-- tipoviv: long
|-- anedi: long
|-- anoanedi: long
|-- fedi: long
|-- tamtoho: long
|-- densidadvi: double
|-- tipoho: long
|-- nacho: long
|-- nucleofam: long
|-- nhijopar: string
|-- nhijo: string
|-- nhijomenor: string

In [6]:
glueContext.write_dynamic_frame.from_options(
       frame = echhogares_res,
       connection_type = "s3",
       connection_options = {"path": "s3://mitramiss/ECHHogares_parquet"},
       format = "parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

To put all the history data into a single file, we need to convert it to a data frame, repartition it, and
write it out.